In [10]:
import pandas as pd
import requests
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# import scikit-learn as sklearn


In [7]:
docs_url = "https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json"
# https://github.com/alexeygrigorev/llm-rag-workshop/blob/main/notebooks/documents.json
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []
for course in documents_raw:
    course_name = course['course']
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
df = pd.DataFrame(documents)
df.head()
df = df [['course','section','question','text']]

In [9]:
#Converting text to vector
from sklearn.feature_extraction.text import CountVectorizer
cv_og = CountVectorizer(min_df = 5)
cv_og.fit(df.text)
cv_og.get_feature_names_out().shape

# #smaller set of documents
X_og = cv_og.transform(df.text)
df_term_matrix = pd.DataFrame(X_og.toarray(), columns=cv_og.get_feature_names_out())
df_term_matrix.head(15)

,01,02,03,04,05,06,09,10,100,11,...,yellow_tripdata_2021,yes,yet,yml,you,your,yourself,youtube,zip,zoomcamp
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,2,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,1,0,0,0,2
6,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,3,2,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,3,1,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,1,0,0


In [7]:
documents_smaller = [
    'Course starts on 15th Jan 2024',
    'Prerequisites listed on GitHub',
    'Submit Homeworks after start date',
    'Registration not required for participation',
    'SEtup Google Cloud and Python before course'
]
cv = CountVectorizer(stop_words='english')
cv.fit(documents_smaller)
cv.get_feature_names_out().shape
X  = cv.transform(documents_smaller)
pd.DataFrame(X.todense(), columns = cv.get_feature_names_out())

,15th,2024,after,and,before,cloud,course,date,for,github,...,on,participation,prerequisites,python,registration,required,setup,start,starts,submit
0,1,1,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,1,...,1,0,1,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
3,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,1,1,0,0,0,0
4,0,0,0,1,1,1,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0


In [10]:
#replacing countvectorizer with tfidfvectorizer
#Converting text to vector
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf_cv_og = TfidfVectorizer(min_df = 5)
tf_idf_cv_og.fit(df.text)
tf_idf_cv_og.get_feature_names_out().shape

# #smaller set of documents
tf_idf_X_og = tf_idf_cv_og.transform(df.text)
tf_idf_df_term_matrix = pd.DataFrame(tf_idf_X_og.toarray(), columns=tf_idf_cv_og.get_feature_names_out())
tf_idf_df_term_matrix.head(15)

,01,02,03,04,05,06,09,10,100,11,...,yellow_tripdata_2021,yes,yet,yml,you,your,yourself,youtube,zip,zoomcamp
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.428961
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.201211,0.0,0.0,0.152763,0.000000,0.000000,0.000000,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.250262,0.000000,0.000000,0.000000,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.140198,0.000000,0.000000,0.000000,0.0,0.000000
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.067261,0.045980,0.000000,0.000000,0.0,0.151170
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.186762,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.150641,0.0,0.0,0.171553,0.156367,0.000000,0.000000,0.0,0.000000
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.168628,0.0,...,0.0,0.128897,0.0,0.0,0.146791,0.066898,0.000000,0.000000,0.0,0.000000
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.085284,0.000000,0.000000,0.127322,0.0,0.000000


In [49]:
query_1 = "Do I need to know python to sign up for the January course?"
query_2 = "I just discovered the course, is it too late to join?"
q = tf_idf_cv_og.transform([query_2])
q.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [50]:
query_zip_value = dict(zip(tf_idf_cv_og.get_feature_names_out(),q.toarray()[0]))
tf_idf_doc_zip_value =  dict(zip(tf_idf_cv_og.get_feature_names_out(),tf_idf_X_og.toarray()[0]))

In [51]:
dist_dot_product = tf_idf_X_og.dot(q.T).toarray()
print (dist_dot_product[:5])
from sklearn.metrics.pairwise import cosine_similarity
dist_cosine_similarity = cosine_similarity(tf_idf_X_og, q)
print (dist_cosine_similarity[:5])
print (dist_cosine_similarity.shape)
dist_cosine_similarity = dist_cosine_similarity.flatten()
import numpy as np
np.argsort(dist_cosine_similarity)[-5:]

[[0.40952467]
 [0.        ]
 [0.02980102]
 [0.21054948]
 [0.02084895]]
[[0.40952467]
 [0.        ]
 [0.02980102]
 [0.21054948]
 [0.02084895]]
(948, 1)


array([ 22, 449, 448, 440,   0])

In [53]:
df.iloc[0]

course                              data-engineering-zoomcamp
section                      General course-related questions
question                 Course - When will the course start?
text        The purpose of this document is to capture fre...
Name: 0, dtype: object

In [73]:
matrices = {}
vectorizers = {}
fields = ['section','question','text']
for f in fields:
    cv_fields = TfidfVectorizer(stop_words='english', min_df = 5)
    X_fields = cv_fields.fit_transform(df[f])
    matrices[f]=X_fields
    vectorizers[f]=cv_fields


In [62]:
matrices

{'section': <948x67 sparse matrix of type '<class 'numpy.float64'>'
 	with 3094 stored elements in Compressed Sparse Row format>,
 'question': <948x562 sparse matrix of type '<class 'numpy.float64'>'
 	with 4333 stored elements in Compressed Sparse Row format>,
 'text': <948x2118 sparse matrix of type '<class 'numpy.float64'>'
 	with 26463 stored elements in Compressed Sparse Row format>}

In [63]:
query_2

'I just discovered the course, is it too late to join?'

In [101]:
score = np.zeros(len(df))
boosts = {
    'question': 3
}
for f in fields:
    print (f)
    print (vectorizers[f])

    query_field = vectorizers[f].transform([query_2])
    X_field = matrices[f]
    cosine_similarity_field = cosine_similarity(X_field, query_field).flatten()
    boost_fields = boosts.get(f, 1.0)
    score = score + boost_fields * cosine_similarity_field

section
TfidfVectorizer(min_df=5, stop_words='english')
question
TfidfVectorizer(min_df=5, stop_words='english')
text
TfidfVectorizer(min_df=5, stop_words='english')


In [86]:
idx = np.argsort(score)[-5:]
df.iloc[idx]

,course,section,question,text
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
9,data-engineering-zoomcamp,General course-related questions,Course - Which playlist on YouTube should I re...,All the main videos are stored in the Main “DA...
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
7,data-engineering-zoomcamp,General course-related questions,Course - Can I follow the course after it fini...,"Yes, we will keep all the materials after the ..."
448,machine-learning-zoomcamp,General course-related questions,I’m new to Slack and can’t find the course cha...,Here’s how you join a in Slack: https://slack....


In [102]:
#filtering by courser
filters = {
    'course': 'data-engineering-zoomcamp'
}

In [109]:
for key,value in filters.items():
    mask = (df[key] == value).astype(int).values
    score_field = score * mask
score_field
idx = np.argsort(-score_field)[:5]
df.iloc[idx]

,course,section,question,text
7,data-engineering-zoomcamp,General course-related questions,Course - Can I follow the course after it fini...,"Yes, we will keep all the materials after the ..."
0,data-engineering-zoomcamp,General course-related questions,Course - When will the course start?,The purpose of this document is to capture fre...
1,data-engineering-zoomcamp,General course-related questions,Course - What are the prerequisites for this c...,GitHub - DataTalksClub data-engineering-zoomca...
4,data-engineering-zoomcamp,General course-related questions,Course - What can I do before the course starts?,You can start by installing and setting up all...
5,data-engineering-zoomcamp,General course-related questions,Course - how many Zoomcamps in a year?,"There are 3 Zoom Camps in a year, as of 2024. ..."


In [98]:
p = np.array([1,4,5,7,16,3,8,2])
np.argsort(p)[-4:]

array([2, 3, 6, 4])

In [99]:
np.argsort(-p)[:4]

array([4, 6, 3, 2])

In [100]:
np.argsort(p)

array([0, 7, 5, 1, 2, 3, 6, 4])

In [23]:
class TextSearch:

    def __init__(self,text_fields):
        self.text_fields = text_fields
        self.matrices = {}
        self.vectorizers = {}

    def fit(self, records, vectorizer_params = {}):
        self.df = pd.DataFrame(records)

        for f in self.text_fields:
            cv = TfidfVectorizer(**vectorizer_params)
            X = cv.fit_transform(self.df[f])
            self.matrices[f] = X
            self.vectorizers[f] = cv

    def search(self, query, n_results = 10, boost = {}, filter={}):
        score = np.zeros(len(self.df))
        for f in self.text_fields:
            boost_field = boost.get(f,1.0)
            query_field= self.vectorizers[f].transform([query])
            print (self.matrices[f].shape)
            print ('query_field',query_field.shape)
            cosine_field = cosine_similarity(self.matrices[f],query_field).flatten()
            print ('cosine_field',cosine_field.shape)
            # print ('boost_field',boost_field.shape)
            print ('score_earlier',score.shape)
            score = score + boost_field * cosine_field
            print ('score',score.shape)
        for key,value in filter.items():
            mask = (self.df[key]==value).values
            score = score * mask

        idx = np.argsort(-score)[:n_results]
        print (idx)
        results= self.df.iloc[idx]
        return results.to_dict(orient='records')







In [24]:
index = TextSearch(
    text_fields = ['question','section','text']
)
index.fit(documents)
index.search(
    query= "I just signed up. Is it too late to join the course?",
    n_results= 5,
    boost={'question':3.0},
    filter={'course':'data-engineering-zoomcamp'}

)

(948, 2051)
query_field (1, 2051)
cosine_field (948,)
score_earlier (948,)
score (948,)
(948, 86)
query_field (1, 86)
cosine_field (948,)
score_earlier (948,)
score (948,)
(948, 6711)
query_field (1, 6711)
cosine_field (948,)
score_earlier (948,)
score (948,)
[ 2  0  7 15  4]


[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineerin

In [25]:
#Applying SVD to the X matrix
X

NameError: name 'X' is not defined

In [ ]:
from sklearn.decomposition import TruncatedSVD